In [5]:
project      = 'hoshinolab_proteomics_age'
from bin import nimb_link
NIMB_HOME = nimb_link.link_with_nimb()
from setup.get_vars import Get_Vars, SetProject
from distribution import utilities as utils
from stats.db_processing import Table
from stats.preprocessing import Preprocess
from stats import preprocessing, varia
from stats.make_stats_grid import MakeGrid
from stats.stats_models import ANOVA_do
from stats.plotting import Make_Plot_Regression, Make_plot_group_difference
tab = Table()

all_vars = Get_Vars()
project_vars = all_vars.projects[project]

getvars      = Get_Vars()
nimb_stats   = getvars.stats_vars
projects     = getvars.projects
NIMB_tmp     = getvars.location_vars['local']['NIMB_PATHS']['NIMB_tmp']
fname_groups = projects[project]["fname_groups"]
nimb_stats   = SetProject(NIMB_tmp, nimb_stats, project, fname_groups).stats
project_vars = projects[project]
stats_paths  = nimb_stats['STATS_PATHS']

def get_X_data_per_group_all_groups(group):
    # extract X_scaled values for the brain parameters
        predicted_target = project_vars["prediction_target"]
        if not predicted_target:
            predicted_target = project_vars["group_col"]
        if group == 'all':
                df_clin_group = df_user_stats
                df_X          = df_adjusted
                y_labeled     = preprocessing.label_y(df_user_stats, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        else:
                df_group      = tab.get_df_per_parameter(df_final_grid, project_vars['group_col'], group)
                df_clin_group = tab.rm_cols_from_df(df_group, cols_X)
                df_X          = tab.rm_cols_from_df(df_group, [i for i in df_group.columns.tolist() if i not in cols_X])
                y_labeled     = preprocessing.label_y(df_group, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        return df_X, y_labeled, X_scaled, df_clin_group

/home/ssp/Dropbox/github/nimb/nimb
        credentials are located at: /home/ssp/nimb
        credentials are located at: /home/ssp/nimb


In [7]:
#project_vars

In [10]:
df_user_stats, df_final_grid,\
            df_adjusted,\
            cols_X,\
            groups = MakeGrid(project_vars,
                                nimb_stats).grid()

    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/stats_src.xlsx,
        sheet: 0
    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/grid.csv,
        sheet: 0


In [11]:
group = 'all'
df_X, y_labeled, X_scaled, df_clin_group = get_X_data_per_group_all_groups(group)
features = cols_X

PowerTransformer()


/home/ssp/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3234: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [4]:
sig_cols = ANOVA_do(df_final_grid,
                    project_vars['variables_for_glm'], features,
                    varia.get_dir(stats_paths['anova']),
                    p_thresh = 0.05, intercept_thresh = 0.05).sig_cols

    analysing 537 features for parameter: Age
    analysing 537 features for parameter: Gender
    analysing 537 features for parameter: Scol
{'Age': {'Age': {'pvalues': 0.0, 'intercept': 0.0005824107978201938, 'meas': 'Age', 'struct': 'Age'}, 'Brixton_Brut': {'pvalues': 0.00023828349692281154, 'intercept': 0.022205668123518842, 'meas': 'Brixton_Brut', 'struct': 'Brixton_Brut'}, 'Brixton_Stand': {'pvalues': 0.0004514056135503166, 'intercept': 5.937318387253673e-07, 'meas': 'Brixton_Stand', 'struct': 'Brixton_Stand'}, 'FA_corrected_1back': {'pvalues': 0.0014264282015220527, 'intercept': 0.01123537854410115, 'meas': 'FA_corrected_1back', 'struct': 'FA_corrected_1back'}, 'Hit_FA_corrected_1back': {'pvalues': 0.00776038705922733, 'intercept': 1.3646403699904348e-08, 'meas': 'Hit_FA_corrected_1back', 'struct': 'Hit_FA_corrected_1back'}, 'Hit_FA_corrected_2back': {'pvalues': 0.008336767529196747, 'intercept': 3.706749569788011e-07, 'meas': 'Hit_FA_corrected_2back', 'struct': 'Hit_FA_correcte

In [6]:
Make_Plot_Regression(df_final_grid,
                    sig_cols, project_vars['group_col'],
                    varia.get_dir(stats_paths['simp_lin_reg_dir']))

In [25]:
Make_plot_group_difference(df_final_grid,
                            sig_cols, project_vars['group_col'], groups,
                            varia.get_dir(stats_paths['anova']))